In [1]:
import os
import pickle
import cv2
from keras_facenet import FaceNet
from sklearn.metrics.pairwise import cosine_similarity

2025-05-31 18:46:48.318428: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-31 18:46:48.328883: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748697408.341234   34487 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748697408.345256   34487 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748697408.354537   34487 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
embedder = FaceNet()

I0000 00:00:1748697410.606215   34487 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1923 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [3]:
embedding_dir = r"/home/phoenix/ProjDD/Projects/P05_MobileNet_FaceRecognition/embeddings"

In [4]:
def load_embeddings():
    embeddings = {}
    for filename in os.listdir(embedding_dir):
        if filename.endswith(".pkl"):
            label = filename.split(".")[0]
            with open(os.path.join(embedding_dir, filename), "rb") as f:
                embeddings[label] = pickle.load(f)
    return embeddings

In [5]:
def recognize_face(frame, known_embeddings):
    detections = embedder.extract(frame, threshold=0.95)
    recognized_faces = []

    for detection in detections:
        embedding = detection['embedding']
        box = detection['box']
        similarities = {
            label: cosine_similarity([embedding], [known_embedding])[0][0]
            for label, known_embedding in known_embeddings.items()
        }

        best_match = max(similarities, key=similarities.get)
        if similarities[best_match] > 0.8:
            recognized_faces.append((best_match, box))
        else:
            recognized_faces.append(("Unknown", box))
    
    return recognized_faces

In [6]:
cap = cv2.VideoCapture(0)

In [7]:
def main():
    known_embeddings = load_embeddings()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        recognized_faces = recognize_face(frame, known_embeddings)

        for label, (x, y, width, height) in recognized_faces:
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        cv2.imshow('Real-Time Face Recognition', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

In [8]:

if __name__ == "__main__":
    try:
        main()
    finally:
        cap.release()
        cv2.destroyAllWindows()

2025-05-31 18:46:58.002575: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [0,48,48,3] vs. [1,1,1,32]
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/phoenix/ProjDD/MLJ/Machine_learning/ml/lib/python3.11/site-packages/cv2/qt/plugins"
2025-05-31 18:46:58.214580: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [0,48,48,3] vs. [1,1,1,32]
2025-05-31 18:46:58.504568: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [0,48,48,3] vs. [1,1,1,32]
2025-05-31 18:46:59.016044: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [0,24,24,3] vs. [1,1,1,28]
2025-05-31 18:46:59.997946: I tensorflow/core/framework/local_rendezvous.cc:407] Local 

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


I0000 00:00:1748697432.874028   36383 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


2025-05-31 18:47:23.117460: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [0,48,48,3] vs. [1,1,1,32]


KeyboardInterrupt: 